## Fiber Photometry PSTH Analysis

This notebook:

This notebook aligns ΔF/F photometry traces to behavioral or stimulation events,
computes peri-event averages, applies baseline normalization and z-scoring,
and generates trial-averaged summaries.

Input files should be placed in:
data/raw/photometry/

# Tested with Python 3.10
# Required packages: numpy, pandas, matplotlib, scipy


In [1]:
#CONFIG

In [10]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

DATA_DIR = Path("data/raw/photometry")
dff_files = [DATA_DIR / f"dff_{i}.csv" for i in range(1,5)] #Range according to the amount of animals/files 
event_files = [DATA_DIR / f"{i}_events.csv" for i in range(1,5)]


In [11]:
# --- Cell 1: build ΔF/F from Doric “signal.csv” with robust 405→470 fit + QC ---

def _safe_linfit(x, y, max_iter=3, z=3.0):
    """
    Robust-ish linear fit y ~ a + b*x with iterative outlier trimming (|resid| > z*SD).
    Returns slope, intercept.
    """
    x = np.asarray(x); y = np.asarray(y)
    m = np.isfinite(x) & np.isfinite(y)
    x = x[m]; y = y[m]
    A = np.vstack([x, np.ones_like(x)]).T
    for _ in range(int(max_iter)):
        slope, intercept = np.linalg.lstsq(A, y, rcond=None)[0]
        yhat = intercept + slope * x
        resid = y - yhat
        s = np.nanstd(resid)
        if not np.isfinite(s) or s == 0:
            break
        keep = np.abs(resid) <= float(z) * s
        if keep.sum() < 10:
            break
        A = A[keep]; x = x[keep]; y = y[keep]
    slope, intercept = np.linalg.lstsq(A, y, rcond=None)[0]
    return float(slope), float(intercept)

def preprocess_signal(file_path, max_iter=3, z=3.0, eps_frac=0.01):
    """
    Reads a Doric-exported CSV with columns [time, 470, 405] (already demodulated),
    performs a robust linear fit of 405→470, and computes ΔF/F = (470 - fit(405)) / fit(405).

    Returns: time(s), dFF (np.array), meta (dict of QC/params).
    """
    # Force Pandas to parse all at once to avoid DtypeWarning
    df = pd.read_csv(file_path, header=None, low_memory=False)
    if df.shape[1] < 3:
        raise ValueError(f"{file_path} needs ≥3 columns: time, Ca(470), iso(405).")
    df = df.iloc[:, :3]
    df.columns = ["time","ca","iso"]

    # Coerce to numeric; drop bad time rows only; keep alignment and interpolate signals
    df["time"] = pd.to_numeric(df["time"], errors="coerce")
    df["ca"]   = pd.to_numeric(df["ca"],   errors="coerce")
    df["iso"]  = pd.to_numeric(df["iso"],  errors="coerce")

    df = df.dropna(subset=["time"]).reset_index(drop=True)
    df[["ca","iso"]] = df[["ca","iso"]].interpolate(limit_direction="both")

    if not np.all(np.diff(df["time"].values) > 0):
        raise ValueError("Timestamps must be strictly increasing.")
    if df["iso"].std() < 1e-12:
        raise ValueError("Isosbestic has ~zero variance.")

    # Robust fit iso→Ca
    slope, intercept = _safe_linfit(df["iso"].values, df["ca"].values, max_iter=max_iter, z=z)
    fitted = intercept + slope * df["iso"].values

    # Safe denominator (prevents blow-ups if fitted is tiny)
    eps = float(eps_frac) * float(np.nanmedian(fitted))
    denom = np.maximum(fitted, eps)
    dff = (df["ca"].values - fitted) / denom

    # QC
    r = float(np.corrcoef(df["iso"].values, df["ca"].values)[0,1])
    ss_res = float(np.sum((df["ca"].values - fitted)**2))
    ss_tot = float(np.sum((df["ca"].values - np.mean(df["ca"].values))**2))
    r2 = float(1 - ss_res/ss_tot) if ss_tot > 0 else np.nan
    dt = np.median(np.diff(df["time"].values)); Fs = float(1.0/dt) if dt > 0 else np.nan

    meta = {
        "slope": slope, "intercept": intercept,
        "r": r, "r2": r2, "Fs": Fs, "n": int(len(df)),
        "eps": float(eps), "eps_frac": float(eps_frac),
        "robust_max_iter": int(max_iter), "robust_z_thresh": float(z)
    }
    return df["time"].values, dff, meta

def process_files(signal_files, max_iter=3, z=3.0, eps_frac=0.01):
    """
    Runs preprocess_signal() over a list of Doric 'signal.csv' files, plots ΔF/F,
    and saves dff_{i}.csv plus dff_{i}_qc.json in the same folder.
    """
    for i, signal_file_path in enumerate(signal_files, start=1):
        p = Path(signal_file_path)
        print(f"Processing: {p}")
        try:
            t, dff, meta = preprocess_signal(signal_file_path, max_iter=max_iter, z=z, eps_frac=eps_frac)
        except Exception as e:
            print(f"ERROR on {p.name}: {e}")
            continue

        # Warn if the linear relation is weak (helps catch bad coupling/motion)
        if np.isfinite(meta.get("r2", np.nan)) and meta["r2"] < 0.30:
            print(f"WARNING ({p.name}): low iso→Ca fit (R²={meta['r2']:.2f}). Check motion/bleach or coupling.")

        # Plot (light smoothing ONLY for visualization)
        lo, hi = np.nanpercentile(dff, [1, 99])
        pad = 0.05 * (hi - lo if np.isfinite(hi - lo) and hi > lo else 1.0)
        ylo, yhi = lo - pad, hi + pad
        Fs = meta["Fs"] if np.isfinite(meta["Fs"]) else 30.0
        win = max(3, int(round(Fs * 0.5)))  # ~0.5 s rolling median
        dff_plot = pd.Series(dff).rolling(win, center=True, min_periods=1).median().values

        plt.figure(figsize=(12, 4))
        plt.plot(t, dff_plot, label="ΔF/F (plot-smooth)")
        plt.xlabel("Time (s)"); plt.ylabel("ΔF/F"); plt.title(f"ΔF/F — {p.name}")
        plt.ylim(ylo, yhi); plt.legend(); plt.tight_layout()
        plt.show()

        # Save outputs
        out_dir = os.path.dirname(signal_file_path)
        out_csv = os.path.join(out_dir, f"dff_{i}.csv")
        pd.DataFrame({"Timestamp": t, "dFF": dff}).to_csv(out_csv, index=False)

        out_qc = os.path.join(out_dir, f"dff_{i}_qc.json")
        with open(out_qc, "w") as f:
            json.dump(meta, f, indent=2)

        print(f"Saved: dff_{i}.csv, dff_{i}_qc.json | R²={meta['r2']:.3f}, Fs={meta['Fs']:.2f} Hz")




In [12]:
#Load ΔF/F traces and event onsets; build common peri-stim grid

In [13]:
# ----- I/O -----
def load_preprocessed_dff(dff_file_path):
    df = pd.read_csv(dff_file_path)
    t = pd.to_numeric(df['Timestamp'], errors='coerce').values
    y = pd.to_numeric(df['dFF'], errors='coerce').values
    m = np.isfinite(t) & np.isfinite(y)
    t, y = t[m], y[m]
    order = np.argsort(t)
    return t[order], y[order]

def load_events(file_path):
    ev = pd.read_csv(file_path, header=None)
    et = pd.to_numeric(ev.iloc[:, 1], errors='coerce').dropna().values
    return np.sort(et)

# ----- time grid + interpolation -----
def build_common_grid(timestamps, pre_time, post_time):
    diffs = np.diff(timestamps)
    dt = np.nanmedian(diffs[np.isfinite(diffs)])
    if not np.isfinite(dt) or dt <= 0:
        raise ValueError("Could not infer a valid positive sampling interval from timestamps.")
    # include the right edge
    grid = np.arange(pre_time, post_time + 0.5*dt, dt)
    return grid, dt

def interpolate_trial_to_grid(t_abs, y_abs, event_time, rel_grid, pre_time, post_time):
    t0 = event_time + pre_time
    t1 = event_time + post_time
    m = (t_abs >= t0) & (t_abs <= t1)
    if not np.any(m):
        return None
    t_win = t_abs[m] - event_time     # relative times
    y_win = y_abs[m]
    # need at least two points with some span to interpolate
    if len(t_win) < 2 or (np.nanmax(t_win) - np.nanmin(t_win)) < 1e-9:
        return None
    y_interp = np.interp(rel_grid, t_win, y_win, left=np.nan, right=np.nan)
    return y_interp

# ----- robust stats -----
def mad(x):
    med = np.nanmedian(x)
    return np.nanmedian(np.abs(x - med))

def sem(a, axis=0):
    # Standard Error of the Mean with unbiased std (ddof=1)
    n = np.sum(np.isfinite(a), axis=axis)
    s = np.nanstd(a, axis=axis, ddof=1)
    with np.errstate(invalid='ignore', divide='ignore'):
        return s / np.sqrt(n)


In [14]:

# =========================
# compute + export
# =========================

def compute_measures(t, y, event_times,
                     pre_time=-3.0, post_time=6.0, baseline_end=-0.1,
                     min_baseline_pts=20, robust_scale_factor=1.4826,
                     min_baseline_sec=1.0, min_valid_frac=0.80):
    """
    Compute per recording:
      - baseline-subtracted ΔF/F PSTH (baseline = median over [pre_time, baseline_end])
      - Standard z-score (baseline mean/SD, ddof=1)
      - Robust z-score (baseline median/MAD × 1.4826; falls back to SD if MAD==0)

    Returns:
      rel_grid,
      dff_mean, dff_sem,
      z_std_mean, z_std_sem,
      z_rob_mean, z_rob_sem,
      n_trials_kept_dff, n_trials_kept_zstd, n_trials_kept_zrob
    """
    rel_grid, dt = build_common_grid(t, pre_time, post_time)

    trials_dff   = []  # ΔF/F baseline-subtracted by baseline median
    trials_z_std = []  # (x - mean) / std
    trials_z_rob = []  # (x - median) / (1.4826 * MAD)

    for et in event_times:
        yi = interpolate_trial_to_grid(t, y, et, rel_grid, pre_time, post_time)
        if yi is None:
            continue

        # Baseline on the RELATIVE grid
        bmask = (rel_grid >= pre_time) & (rel_grid <= baseline_end)
        b = yi[bmask]

        # --- Time-based + completeness guards for baseline ---
        required_pts = max(int(np.ceil(min_baseline_sec / dt)), int(min_baseline_pts))
        valid = np.isfinite(b)
        if valid.sum() < required_pts or (valid.sum() / max(b.size, 1)) < min_valid_frac:
            # Skip trials with too little valid baseline
            continue

        b_med  = np.nanmedian(b)
        b_mean = np.nanmean(b)
        b_std  = np.nanstd(b, ddof=1)

        # ΔF/F baseline-subtracted
        trials_dff.append(yi - b_med)

        # Standard z (mean/SD)
        if np.isfinite(b_std) and b_std > 0:
            trials_z_std.append((yi - b_mean) / b_std)

        # Robust z (median/MAD×1.4826). If MAD==0, fall back to SD.
        b_mad = mad(b)
        if np.isfinite(b_mad) and b_mad > 0:
            trials_z_rob.append((yi - b_med) / (robust_scale_factor * b_mad))
        else:
            if np.isfinite(b_std) and b_std > 0:
                trials_z_rob.append((yi - b_med) / b_std)

    # Aggregate -> mean ± SEM across trials
    out = {}

    if len(trials_dff) > 0:
        dff_arr = np.vstack(trials_dff)
        out['dff_mean'] = np.nanmean(dff_arr, axis=0)
        out['dff_sem']  = sem(dff_arr, axis=0)
        out['n_dff']    = dff_arr.shape[0]
    else:
        out['dff_mean'] = out['dff_sem'] = None
        out['n_dff']    = 0

    if len(trials_z_std) > 0:
        zstd_arr = np.vstack(trials_z_std)
        out['z_std_mean'] = np.nanmean(zstd_arr, axis=0)
        out['z_std_sem']  = sem(zstd_arr, axis=0)
        out['n_zstd']     = zstd_arr.shape[0]
    else:
        out['z_std_mean'] = out['z_std_sem'] = None
        out['n_zstd']     = 0

    if len(trials_z_rob) > 0:
        zrob_arr = np.vstack(trials_z_rob)
        out['z_rob_mean'] = np.nanmean(zrob_arr, axis=0)
        out['z_rob_sem']  = sem(zrob_arr, axis=0)
        out['n_zrob']     = zrob_arr.shape[0]
    else:
        out['z_rob_mean'] = out['z_rob_sem'] = None
        out['n_zrob']     = 0

    return (rel_grid,
            out['dff_mean'], out['dff_sem'],
            out['z_std_mean'], out['z_std_sem'],
            out['z_rob_mean'], out['z_rob_sem'],
            out['n_dff'], out['n_zstd'], out['n_zrob'])


def process_files_psth_both(dff_files, event_files,
                            pre_time=-3.0, post_time=6.0, baseline_end=-0.1,
                            min_baseline_pts=20, save_figs=False,
                            min_baseline_sec=1.0, min_valid_frac=0.80):
    """
    For each recording:
      - Plots ΔF/F baseline-subtracted, standard Z, robust Z
      - Saves per-recording CSV with aligned time and mean ± SEM for all three
      - Saves AUC-by-1s-bin CSV for all three
    After all recordings:
      - Saves ONE aggregate CSV with the same time column and one column per recording for each metric
        Column order:
          time_s |
          dFF_mean_rec1..N | dFF_sem_rec1..N |
          z_std_mean_rec1..N | z_std_sem_rec1..N |
          z_robust_mean_rec1..N | z_robust_sem_rec1..N
    """
    per_rec = {}          # i -> dict with rel_t and all measures
    first_out_dir = None  # use this folder for aggregate CSV

    for i, (dff_file_path, event_file_path) in enumerate(zip(dff_files, event_files), start=1):
        if first_out_dir is None:
            first_out_dir = os.path.dirname(dff_file_path)

        print(f"\n=== Recording {i} ===")
        print(f"ΔF/F file: {dff_file_path}")
        print(f"Events  : {event_file_path}")

        t, y = load_preprocessed_dff(dff_file_path)
        et = load_events(event_file_path)

        (rel_t,
         dff_mean, dff_sem,
         zstd_mean, zstd_sem,
         zrob_mean, zrob_sem,
         n_dff, n_zstd, n_zrob) = compute_measures(
            t, y, et,
            pre_time=pre_time, post_time=post_time, baseline_end=baseline_end,
            min_baseline_pts=min_baseline_pts,
            min_baseline_sec=min_baseline_sec, min_valid_frac=min_valid_frac
        )

        # ---- store for later aggregation ----
        per_rec[i] = {
            'rel_t': rel_t,
            'dFF_mean': dff_mean, 'dFF_sem': dff_sem,
            'z_std_mean': zstd_mean, 'z_std_sem': zstd_sem,
            'z_robust_mean': zrob_mean, 'z_robust_sem': zrob_sem
        }

        # ---- Plot ΔF/F baseline-subtracted ----
        if dff_mean is not None:
            plt.figure()
            plt.plot(rel_t, dff_mean, label=f'Rec {i} ΔF/F (baseline-sub)')
            plt.fill_between(rel_t, dff_mean - dff_sem, dff_mean + dff_sem, alpha=0.3)
            plt.axvline(0, color='r', linestyle='--')
            plt.xlabel('Time (s)')
            plt.ylabel('ΔF/F (baseline-subtracted)')
            plt.title(f'PSTH ΔF/F — Rec {i} (n={n_dff} trials)')
            plt.legend()
            plt.tight_layout()
            if save_figs:
                figpath = os.path.join(os.path.dirname(dff_file_path), f'psth_dff_rec{i}.png')
                plt.savefig(figpath, dpi=200)
            plt.show()
        else:
            print("No valid trials for ΔF/F baseline-subtracted.")

        # ---- Plot Standard Z (mean/SD) ----
        if zstd_mean is not None:
            plt.figure()
            plt.plot(rel_t, zstd_mean, label=f'Rec {i} Z (mean/SD)')
            plt.fill_between(rel_t, zstd_mean - zstd_sem, zstd_mean + zstd_sem, alpha=0.25)
            plt.axvline(0, color='r', linestyle='--')
            plt.xlabel('Time (s)')
            plt.ylabel('Z-score (baseline mean/SD)')
            plt.title(f'Standard Z-score — Rec {i} (n={n_zstd} trials)')
            plt.legend()
            plt.tight_layout()
            if save_figs:
                figpath = os.path.join(os.path.dirname(dff_file_path), f'z_std_rec{i}.png')
                plt.savefig(figpath, dpi=200)
            plt.show()
        else:
            print("No valid trials for Standard Z (mean/SD).")

        # ---- Plot Robust Z (median/MAD×1.4826) ----
        if zrob_mean is not None:
            plt.figure()
            plt.plot(rel_t, zrob_mean, label=f'Rec {i} Robust Z (median/MAD×1.4826)')
            plt.fill_between(rel_t, zrob_mean - zrob_sem, zrob_mean + zrob_sem, alpha=0.25)
            plt.axvline(0, color='r', linestyle='--')
            plt.xlabel('Time (s)')
            plt.ylabel('Robust Z-score (median/MAD×1.4826)')
            plt.title(f'Robust Z-score — Rec {i} (n={n_zrob} trials)')
            plt.legend()
            plt.tight_layout()
            if save_figs:
                figpath = os.path.join(os.path.dirname(dff_file_path), f'z_robust_rec{i}.png')
                plt.savefig(figpath, dpi=200)
            plt.show()
        else:
            print("No valid trials for Robust Z (median/MAD).")

        # ---- Export per-recording aligned CSV ----
        out_df = pd.DataFrame({'time_s': rel_t})
        # always include columns; fill with NaN if None
        out_df['dFF_mean'] = dff_mean if dff_mean is not None else np.nan
        out_df['dFF_sem']  = dff_sem  if dff_sem  is not None else np.nan
        out_df['z_std_mean'] = zstd_mean if zstd_mean is not None else np.nan
        out_df['z_std_sem']  = zstd_sem  if zstd_sem  is not None else np.nan
        out_df['z_robust_mean'] = zrob_mean if zrob_mean is not None else np.nan
        out_df['z_robust_sem']  = zrob_sem  if zrob_sem  is not None else np.nan

        export_path = os.path.join(os.path.dirname(dff_file_path), f'psth_and_z_results_rec{i}.csv')
        out_df.to_csv(export_path, index=False)
        print(f"Saved: {export_path}")

        # ---- AUC per 1 s bin for all three (optional but handy) ----
        auc_rows = []
        for start in range(int(pre_time), int(post_time)):
            end = start + 1
            m = (rel_t >= start) & (rel_t < end)

            def auc_or_nan(arr):
                return np.trapz(arr[m], x=rel_t[m]) if arr is not None and np.any(m) else np.nan

            auc_rows.append({
                'TimeBin': f'{start} to {end} s',
                'AUC_dFF': auc_or_nan(dff_mean),
                'AUC_z_std': auc_or_nan(zstd_mean),
                'AUC_z_robust': auc_or_nan(zrob_mean)
            })

        auc_df = pd.DataFrame(auc_rows)
        auc_export_path = os.path.join(os.path.dirname(dff_file_path), f'auc_by_bin_rec{i}.csv')
        auc_df.to_csv(auc_export_path, index=False)
        print(f"Saved: {auc_export_path}")

    # =========================
    # Aggregate CSV (all recs)
    # =========================
    if len(per_rec) == 0:
        print("No recordings to aggregate.")
        return

    # Use the first recording's time grid as the common base
    base_idx = sorted(per_rec.keys())[0]
    base_t = per_rec[base_idx]['rel_t']

    def to_base(arr, t_src, t_dst):
        """Interpolate array onto destination time base; out-of-range -> NaN."""
        if arr is None or t_src is None or len(t_src) == 0:
            return np.full_like(t_dst, np.nan, dtype=float)
        out = np.interp(t_dst, t_src, arr)
        # NaN out-of-range
        out[t_dst < np.nanmin(t_src)] = np.nan
        out[t_dst > np.nanmax(t_src)] = np.nan
        return out

    agg_df = pd.DataFrame({'time_s': base_t})

    metric_order = [
        ('dFF_mean',        'dFF_mean_rec{idx}'),
        ('dFF_sem',         'dFF_sem_rec{idx}'),
        ('z_std_mean',      'z_std_mean_rec{idx}'),
        ('z_std_sem',       'z_std_sem_rec{idx}'),
        ('z_robust_mean',   'z_robust_mean_rec{idx}'),
        ('z_robust_sem',    'z_robust_sem_rec{idx}'),
    ]

    n_recs = len(per_rec)
    for metric_key, col_template in metric_order:
        for idx in range(1, n_recs + 1):
            rel_t = per_rec[idx]['rel_t']
            arr   = per_rec[idx][metric_key]
            agg_df[col_template.format(idx=idx)] = to_base(arr, rel_t, base_t)

    agg_path = os.path.join(first_out_dir, 'psth_z_all_recordings_aligned.csv')
    agg_df.to_csv(agg_path, index=False)
    print(f"Saved aggregate CSV with all recordings: {agg_path}")


In [15]:
# ---- RUN ANALYSIS (requires files in DATA_DIR) ----

In [ ]:
# Recommended baseline guards: ≥1.0 s valid baseline and ≥80% finite samples in that baseline window
process_files_psth_both(
    dff_files,
    event_files,
    pre_time=-3.0,
    post_time=6.0,
    baseline_end=-0.1,
    min_baseline_pts=20,
    min_baseline_sec=1.0,
    min_valid_frac=0.80,
    save_figs=False
)
